# Randomized Optimization




Package to use non gradienty descent for NNW (mlrose, hivemlrose)

implement ro algs on toy: knapsack, traveling salesman, ...

Discuss what works best given the different problems